# Convert Wingtra flight plan to KML

In [ ]:
# Author: Farid Javadnejad
# Date: 2025-02-18
#
# DESCRIPTION:
# This script converts a Wingtra UAS flight plan from a .flightplan file (i.e., JSON) to a KML file, 
# extracting the survey area and adding a Placemark for the takeoff/landing location.
#
# DISCLAIMER:
# This script was developed with the assistance of AI tools for debugging, reviewing, and testing.
# ---------------------------------------------------------------------------------------------------

In [6]:
#pip install simplekml fastkml lxml
import json
import simplekml
import os

In [ ]:
def convert_flightplan_to_kml(file_path):
    # Read the JSON flight plan file
    with open(file_path, "r") as file:
        flightplan = json.load(file)

    # Extract the flight plan polygon (excluding geofence)
    flight_plan = flightplan.get("flightPlan", {})
    survey_polygon = flight_plan.get("items", [])

    # Find the survey area polygon from 'ComplexItem' with type 'area'
    survey_area = None
    for item in survey_polygon:
        if item.get("complexItemType") == "area" and "polygon" in item:
            survey_area = item["polygon"]
            break  # Take only the first survey area

    if not survey_area:
        print("No survey area polygon found.")
        return

    # Convert survey_area to KML-compatible format (tuples of (lon, lat))
    try:
        # Ensure coordinates are formatted as [lon, lat]
        kml_coordinates = [(point[1], point[0]) for point in survey_area]
    except IndexError:
        print("Invalid coordinate format in survey area.")
        return

    # Create a KML file
    kml = simplekml.Kml()

    # Add a polygon for the survey area
    pol = kml.newpolygon(name="Survey Area", outerboundaryis=kml_coordinates)
    pol.style.polystyle.color = simplekml.Color.changealpha("4C", simplekml.Color.rgb(0, 170, 255))  # 30% opacity
    pol.style.linestyle.color = simplekml.Color.rgb(0, 170, 255)  # 100% opacity
    pol.style.linestyle.width = 2.0

    # Add a Placemark for the takeoff/landing location
    takeoff_landing = flight_plan.get("items", [])[0].get("coordinate", [])
    if takeoff_landing:
        kml.newpoint(name="Takeoff/Landing", coords=[(takeoff_landing[1], takeoff_landing[0], takeoff_landing[2])])

    # Save the KML file with the same name as input but with .kml extension
    kml_filename = os.path.splitext(file_path)[0] + ".kml"
    kml.save(kml_filename)
    print(f"KML file saved: {kml_filename}")


In [8]:
read_folder = "C:/Farid/gitProjects/convert_json_to_kml/data"
read_file = "WingtraFlightPlan.flightplan"
file_path = str(read_folder) + "/" + str(read_file)

convert_flightplan_to_kml(file_path)

KML file saved: C:/Farid/gitProjects/convert_json_to_kml/data/WingtraFlightPlan.kml
